In [66]:
import mne 
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
figure(figsize=(1,1))
from mne.decoding import UnsupervisedSpatialFilter
from sklearn.decomposition import PCA, FastICA
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import make_pipeline
from mne.decoding import Vectorizer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import classification_report
import itertools
from sklearn.svm import SVC 

from mne.preprocessing import ICA


In [67]:
setFile = '/Volumes/flash32/EEGLAB/SA20140306/20140306_1133.set'

In [68]:
raw = mne.io.read_raw_eeglab(setFile,preload = True)

Reading /Volumes/flash32/EEGLAB/SA20140306/20140306_1133.fdt
Reading 0 ... 951345  =      0.000 ...  1858.096 secs...


In [69]:
def find_events(raw):
    return mne.find_events(raw,output='offset',
                           min_duration=1/raw.info['sfreq'],
                           shortest_event=0.5,
                           initial_event=False)



In [70]:
events = find_events(raw)
events

5792 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  31 129 130 131 132 133 134 135 136 137 138 139
 140 141 142 143 144 145 155 156 157 158 159 160 161 162 163 164 165]


array([[  6715,      0,    131],
       [  7046,      0,     31],
       [  7049,      0,    137],
       ...,
       [841233,      0,      4],
       [841501,      0,    134],
       [841503,      0,    129]])

In [71]:
def get_trials(events):
    trials = []
    current_trial = []
    for event in events:
        if event[-1] == 31 and current_trial != []:
            trials.append(current_trial)
            current_trial = []
            current_trial.append(event)
        else:
            current_trial.append(event)
    trials.append(current_trial)
    return trials

In [72]:
trials = get_trials(events)

In [73]:
"""
filter out all trials that are not within stim code definitions.
The second flanker stim code should be one plus the first flanker stim code. 
Than concatenates trials back into events.
"""

def filter_trials(trials):
    events = find_events(raw)
    trials = get_trials(events)
    filteredTrials = []
    totalTrials = 0
    filtered = 0
    for x in trials:
        totalTrials += 1
        
    for trial in trials:
        first_stim = False
        third_stim = False
        first_stim_code = 0
        
        for event in trial:
            if first_stim == True and third_stim == True:
                filteredTrials.append(trial)
                filtered += 1
                first_stim = False
                third_stim = False
                break;
            else:    
                #print(event[-1])
                if event[-1] <= 24 and first_stim == False:
                        first_stim_code = event[-1]
                        first_stim = True
                elif event[-1] <=24 and first_stim == True:    
                    if event[-1] + 1 == first_stim_code:
                        third_stim = True
                        first_stim_code = 0
    print('number of trials removed: ',totalTrials - filtered)
    return np.concatenate(filteredTrials,axis = 0)

In [74]:
filter_trials(trials)

5792 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  31 129 130 131 132 133 134 135 136 137 138 139
 140 141 142 143 144 145 155 156 157 158 159 160 161 162 163 164 165]
number of trials removed:  1


array([[  7046,      0,     31],
       [  7049,      0,    137],
       [  7052,      0,    164],
       ...,
       [841233,      0,      4],
       [841501,      0,    134],
       [841503,      0,    129]])

In [75]:
def get_events_with_stim_code(events):
    """
    Return only events with stim codes from all the events
    make sure to make numpy arrays all events and return final array numpy
    """
    stim_events = []
    for event in events:
        if event[-1] <=24:
            stim_events.append(np.array(event))
    return np.array(stim_events)  

In [76]:
get_events_with_stim_code(filter_trials(trials))

5792 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  31 129 130 131 132 133 134 135 136 137 138 139
 140 141 142 143 144 145 155 156 157 158 159 160 161 162 163 164 165]
number of trials removed:  1


array([[  7064,      0,      2],
       [  7314,      0,      6],
       [  7564,      0,      1],
       ...,
       [840706,      0,      5],
       [840965,      0,      3],
       [841233,      0,      4]])

In [77]:
def artifact_removal(eeglab):
    """
    NOTE: use this method after creating epoch objects to speed up process!
    
    Description:
        Perform notch filter, frequenc filter, and ICA 
        
    Variables: 
        eeglab: mne.io.eeglab.eeglab.RawEEGLAB object
    returns:
        eeglab: mne.io.eeglab.eeglab.RawEEGLAB object

    """
    
    raw.notch_filter(np.arange(60,241,60), filter_length='auto', phase = 'zero')
    raw.filter(5,100, fir_design='firwin')
    raw.resample(256, n_jobs=10)
    ica = ICA(n_components=None, max_iter = 1000, random_state = 42)
    ica.fit(raw,picks = None, decim=3, reject=dict(mag=4e-12, grad=4000e-13))
    ica.detect_artifacts(raw,start_find=None, stop_find=None, ecg_ch=None, 
                 ecg_score_func='pearsonr', skew_criterion=None, 
                 kurt_criterion=None, var_criterion=None,add_nodes=None)
    
    return ica.apply(raw)

In [78]:
new_stim_code_event_ids = {
     
    "b-f2-wrd":1,
    "b-f1-wrd":2,
    "b-tg-wrd":3,

    "b-f2-spk":4,
    "b-f1-spk":5,
    "b-tg-spk":6,

    "b-f2-pic":7,
    "b-f1-pic":8,
    "b-tg-pic":9,

    "b-f2-snd":10,
    "b-f1-snd":11,
    "b-tg-snd":12,

    "f-f2-wrd":13,
    "f-f1-wrd":14,
    "f-tg-wrd":15,

    "f-f2-spk":16,
    "f-f1-spk":17,
    "f-tg-spk":18,

    "f-f2-pic":19,
    "f-f1-pic":20,
    "f-tg-pic":21,

    "f-f2-snd":22,
    "f-f1-snd":23,
    "f-tg-snd":24
}

In [79]:
def create_epochs(raw,events,event_id,):
    tmax = (1/raw.info['sfreq']) * 499  #to get desired epoch length
    return mne.Epochs(raw=raw,events=events,event_id=event_id,tmin=0,tmax=tmax,preload=True )

In [80]:
def remove_artifacts_and_create_epochs(raw):
    events = filter_trials(trials)
    stim_code_events = get_events_with_stim_code(events)
    #stim_code_event_ids = get_stim_code_event_ids(stim_code_events,new_stim_code_event_ids)
    raw_ica = artifact_removal(raw)
    
    return create_epochs(raw_ica,events,new_stim_code_event_ids)

In [81]:
epoch = remove_artifacts_and_create_epochs(raw)

5792 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  31 129 130 131 132 133 134 135 136 137 138 139
 140 141 142 143 144 145 155 156 157 158 159 160 161 162 163 164 165]
number of trials removed:  1
Setting up band-stop filter
Filter length of 3379 samples (6.600 sec) selected
Setting up band-pass filter from 5 - 1e+02 Hz
l_trans_bandwidth chosen to be 2.0 Hz
h_trans_bandwidth chosen to be 25.0 Hz
Filter length of 845 samples (1.650 sec) selected
5792 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  31 129 130 131 132 133 134 135 136 137 138 139
 140 141 142 143 144 145 155 156 157 158 159 160 161 162 163 164 165]


[Parallel(n_jobs=10)]: Done  52 tasks      | elapsed:   17.2s
[Parallel(n_jobs=10)]: Done 129 out of 129 | elapsed:   46.0s finished


5388 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  31 129 130 131 132 133 134 135 136 137 138 139
 140 141 142 143 144 145 155 156 157 158 159 160 161 162 163 164 165]
Fitting ICA to data using 128 channels (please be patient, this may take a while)
Inferring max_pca_components from picks
Using all PCA components: 128
Fitting ICA took 146.7s.
    Searching for artifacts...
Artifact indices found:
    
Ready.
Transforming to ICA space (128 components)
Zeroing out 0 ICA components
1920 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 1920 events and 500 original time points ...
936 bad epochs dropped


In [48]:
type(epoch)

mne.epochs.Epochs

In [83]:
epoch.get_data().shape

(984, 129, 500)

In [49]:
epoch.events

array([[  7064,      0,      2],
       [  7314,      0,      6],
       [  7564,      0,      1],
       ...,
       [474286,      0,     23],
       [474536,      0,     18],
       [474787,      0,     22]])

In [50]:
epoch.event_id

{'b-f1-pic': 8,
 'b-f1-snd': 11,
 'b-f1-spk': 5,
 'b-f1-wrd': 2,
 'b-f2-pic': 7,
 'b-f2-snd': 10,
 'b-f2-spk': 4,
 'b-f2-wrd': 1,
 'b-tg-pic': 9,
 'b-tg-snd': 12,
 'b-tg-spk': 6,
 'b-tg-wrd': 3,
 'f-f1-pic': 20,
 'f-f1-snd': 23,
 'f-f1-spk': 17,
 'f-f1-wrd': 14,
 'f-f2-pic': 19,
 'f-f2-snd': 22,
 'f-f2-spk': 16,
 'f-f2-wrd': 13,
 'f-tg-pic': 21,
 'f-tg-snd': 24,
 'f-tg-spk': 18,
 'f-tg-wrd': 15}

In [61]:
#epoch.save(fname='20140306_1133-epo.fif')

In [84]:
data = epoch.get_data()

In [94]:
data.shape

(984, 129, 500)

In [101]:
def getAverageVar(data,segmentNum,epochNum):
    
    

for channel in data[0]:
    max_arr.append(channel[0:50])
    arr = np.array(max_arr)
    print(arr.shape)

(1, 50)
(2, 50)
(3, 50)
(4, 50)
(5, 50)
(6, 50)
(7, 50)
(8, 50)
(9, 50)
(10, 50)
(11, 50)
(12, 50)
(13, 50)
(14, 50)
(15, 50)
(16, 50)
(17, 50)
(18, 50)
(19, 50)
(20, 50)
(21, 50)
(22, 50)
(23, 50)
(24, 50)
(25, 50)
(26, 50)
(27, 50)
(28, 50)
(29, 50)
(30, 50)
(31, 50)
(32, 50)
(33, 50)
(34, 50)
(35, 50)
(36, 50)
(37, 50)
(38, 50)
(39, 50)
(40, 50)
(41, 50)
(42, 50)
(43, 50)
(44, 50)
(45, 50)
(46, 50)
(47, 50)
(48, 50)
(49, 50)
(50, 50)
(51, 50)
(52, 50)
(53, 50)
(54, 50)
(55, 50)
(56, 50)
(57, 50)
(58, 50)
(59, 50)
(60, 50)
(61, 50)
(62, 50)
(63, 50)
(64, 50)
(65, 50)
(66, 50)
(67, 50)
(68, 50)
(69, 50)
(70, 50)
(71, 50)
(72, 50)
(73, 50)
(74, 50)
(75, 50)
(76, 50)
(77, 50)
(78, 50)
(79, 50)
(80, 50)
(81, 50)
(82, 50)
(83, 50)
(84, 50)
(85, 50)
(86, 50)
(87, 50)
(88, 50)
(89, 50)
(90, 50)
(91, 50)
(92, 50)
(93, 50)
(94, 50)
(95, 50)
(96, 50)
(97, 50)
(98, 50)
(99, 50)
(100, 50)
(101, 50)
(102, 50)
(103, 50)
(104, 50)
(105, 50)
(106, 50)
(107, 50)
(108, 50)
(109, 50)
(110, 50)
(111, 50

In [112]:
firstEpoch50data = np.array(max_arr)
firstEpoch50data.shape

(129, 50)

In [119]:
mean = np.mean(firstEpoch50data)
mean

2.6014002251925666e-09

In [120]:
mean - firstEpoch50data[0]

array([ 2.60140023e-09,  3.21936341e-06,  1.71719606e-05, -6.18284394e-07,
       -2.52973396e-05, -3.19022771e-05, -4.93652826e-05, -4.72085403e-05,
       -3.53708711e-05, -1.63027217e-05,  8.80742258e-06,  1.42474809e-05,
        2.02112505e-05,  2.43080973e-05,  1.51388184e-05,  5.70301917e-06,
        8.61577987e-06,  1.03310416e-05,  1.58864671e-05,  1.60469165e-05,
        1.92008919e-05,  3.15338995e-05,  1.93752904e-05,  8.32157437e-06,
        8.30281535e-06,  1.78685547e-05,  1.26299024e-05, -2.51697418e-06,
        4.35120269e-06,  8.66018257e-06,  1.91083911e-05,  1.94131771e-05,
        1.55417799e-05,  1.50397073e-05,  6.64227585e-06,  5.08770678e-06,
        5.09859605e-06,  7.28987875e-06,  9.61881380e-06,  1.01314268e-05,
        7.27203135e-06,  1.09908318e-05,  1.01266422e-05,  1.33903364e-05,
        2.17708593e-05,  1.29925540e-05,  1.26062540e-05,  1.44216904e-05,
        1.02608729e-05,  1.37982195e-05])

In [118]:
firstEpoch50data

array([[ 0.00000000e+00, -3.21676201e-06, -1.71693592e-05, ...,
        -1.44190890e-05, -1.02582715e-05, -1.37956181e-05],
       [ 0.00000000e+00,  1.67594076e-07, -5.33031974e-06, ...,
        -1.38133314e-05, -8.47172674e-06, -1.42941024e-05],
       [ 0.00000000e+00, -2.04089828e-06, -5.93303656e-06, ...,
        -1.65377441e-05, -1.36941415e-05, -1.27385043e-05],
       ...,
       [ 0.00000000e+00,  5.13467936e-07,  1.06802868e-05, ...,
         3.05669011e-07, -2.39182552e-06, -2.65470145e-06],
       [ 0.00000000e+00, -2.53321829e-07,  6.58559300e-06, ...,
        -3.19511373e-06, -6.79651916e-06, -4.07718894e-06],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])